In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recommendation-system/movies.csv
/kaggle/input/movie-recommendation-system/ratings.csv


# Movie Recommendation using KNN 

In [2]:
movies_df=pd.read_csv('/kaggle/input/movie-recommendation-system/movies.csv')
ratings_df=pd.read_csv('/kaggle/input/movie-recommendation-system/ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_df.shape

(62423, 3)

In [4]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [7]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [8]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [9]:
ratings_df.shape

(25000095, 4)

In [10]:
df=movies_df.merge(ratings_df)
df1=df.drop(['genres', 'timestamp'], axis=1)
df1.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),2,3.5
1,1,Toy Story (1995),3,4.0
2,1,Toy Story (1995),4,3.0
3,1,Toy Story (1995),5,4.0
4,1,Toy Story (1995),8,4.0


In [11]:
df2=df1.groupby('title')['rating'].count().reset_index().rename(columns={'rating': 'ratingcount'})
df2.head()

,title,ratingcount
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",179
2,#1 Cheerleader Camp (2010),9
3,#Captured (2017),2
4,#Female Pleasure (2018),3


In [12]:
df3=df1.merge(df2,on='title',how='left')
df3.head()

,movieId,title,userId,rating,ratingcount
0,1,Toy Story (1995),2,3.5,57309
1,1,Toy Story (1995),3,4.0,57309
2,1,Toy Story (1995),4,3.0,57309
3,1,Toy Story (1995),5,4.0,57309
4,1,Toy Story (1995),8,4.0,57309


In [13]:
print(f'10% of count= {df3.ratingcount.quantile(.1)}')
print(f'25% of count= {df3.ratingcount.quantile(.25)}')
print(f'50% of count= {df3.ratingcount.quantile(.5)}')
print(f'75% of count= {df3.ratingcount.quantile(.75)}')
print(f'85% of count= {df3.ratingcount.quantile(.85)}')
print(f'100% of count= {df3.ratingcount.quantile(1)}')

10% of count= 825.0
25% of count= 2986.0
50% of count= 9152.0
75% of count= 20757.0
85% of count= 30910.0
100% of count= 81491.0


In [14]:
df4=df3[df3.ratingcount>=10000]
df4.shape

(11878413, 5)

In [15]:
movie_rating_pivote=df4.pivot_table(index='title',columns='userId', values='rating').fillna(0)
movie_rating_pivote.head()

userId,1,2,3,4,5,6,7,8,9,10,...,162532,162533,162534,162535,162536,162537,162538,162539,162540,162541
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,2.5,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,5.0,4.0,0.0,4.0,0.0,5.0,3.0,4.5,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
28 Days Later (2002),0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

movie_rating_df=csr_matrix(movie_rating_pivote.values)

model_knn=NearestNeighbors(metric='cosine', algorithm='brute',n_neighbors=11)
model_knn.fit(movie_rating_df)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=11)

In [17]:
movie_rating_df.shape

(588, 162109)

In [18]:
distance,indices=model_knn.kneighbors(movie_rating_pivote.iloc[np.random.choice(movie_rating_df.shape[0]),:].values.reshape(1,-1),n_neighbors=11)

for i in range(0, len(distance.flatten())):
    if i == 0:
        print(f'Recommendation for a movie: {movie_rating_pivote.index[indices.flatten()[i]]}')
    else:
        print(f'{i}: {movie_rating_pivote.index[indices.flatten()[i]]} with a distance {distance.flatten()[i]}')

Recommendation for a movie: Sense and Sensibility (1995)
1: Four Weddings and a Funeral (1994) with a distance 0.5835726680016693
2: Dead Man Walking (1995) with a distance 0.600731202934407
3: Much Ado About Nothing (1993) with a distance 0.6008205228350088
4: Piano, The (1993) with a distance 0.6132756605912677
5: Postman, The (Postino, Il) (1994) with a distance 0.6245163236472786
6: Leaving Las Vegas (1995) with a distance 0.6262457702731282
7: Like Water for Chocolate (Como agua para chocolate) (1992) with a distance 0.6301102213639552
8: Birdcage, The (1996) with a distance 0.6306896054016372
9: Mr. Holland's Opus (1995) with a distance 0.6313555177764647
10: Truth About Cats & Dogs, The (1996) with a distance 0.6322794831134104
